In [ ]:
conda install -c conda-forge pandas-profiling=2.9.0
conda update -n base -c defaults conda

In [60]:
import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from pandas_profiling import ProfileReport
from math import ceil

sns.set()

In [61]:
donors=pd.read_csv("donors.csv", sep=",", header=0, dtype={"NOEXCH": str,"RDATE_5":str}) #Due to a DytpeWarning, it's preferred to specify which is the type we are dealing with
donors

,Unnamed: 0,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,...,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,0,2009-01-01,GRI,0,IL,61081,,,1957-12-01,0,...,7.741935,95515,0,L,4,E,X,X,X,C
1,1,2014-01-01,BOA,1,CA,91326,,,1972-02-01,0,...,15.666667,148535,0,L,2,G,X,X,X,A
2,2,2010-01-01,AMH,1,NC,27017,,,NaN,0,...,7.481481,15078,1,L,4,E,X,X,X,C
3,3,2007-01-01,BRY,0,CA,95953,,,1948-01-01,0,...,6.812500,172556,1,L,4,E,X,X,X,C
4,4,2006-01-01,,0,FL,33176,,,1940-01-01,0,...,6.864865,7112,1,L,2,F,X,X,X,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,95407,2016-01-01,ASE,1,AK,99504,,,NaN,0,...,25.000000,184568,0,L,1,G,X,X,X,C
95408,95408,2016-01-01,DCD,1,TX,77379,,,1970-01-01,0,...,20.000000,122706,1,L,1,F,X,X,X,A
95409,95409,2015-01-01,MBC,1,MI,48910,,,1958-01-01,0,...,8.285714,189641,1,L,3,E,X,X,X,B
95410,95410,2006-01-01,PRV,0,CA,91320,,,1960-05-01,0,...,12.146341,4693,1,L,4,F,X,X,X,A


In [62]:
donors.drop("Unnamed: 0",axis=1,inplace=True)
donors

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,2009-01-01,GRI,0,IL,61081,,,1957-12-01,0,,...,7.741935,95515,0,L,4,E,X,X,X,C
1,2014-01-01,BOA,1,CA,91326,,,1972-02-01,0,,...,15.666667,148535,0,L,2,G,X,X,X,A
2,2010-01-01,AMH,1,NC,27017,,,NaN,0,,...,7.481481,15078,1,L,4,E,X,X,X,C
3,2007-01-01,BRY,0,CA,95953,,,1948-01-01,0,,...,6.812500,172556,1,L,4,E,X,X,X,C
4,2006-01-01,,0,FL,33176,,,1940-01-01,0,X,...,6.864865,7112,1,L,2,F,X,X,X,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,2016-01-01,ASE,1,AK,99504,,,NaN,0,,...,25.000000,184568,0,L,1,G,X,X,X,C
95408,2016-01-01,DCD,1,TX,77379,,,1970-01-01,0,,...,20.000000,122706,1,L,1,F,X,X,X,A
95409,2015-01-01,MBC,1,MI,48910,,,1958-01-01,0,,...,8.285714,189641,1,L,3,E,X,X,X,B
95410,2006-01-01,PRV,0,CA,91320,,,1960-05-01,0,X,...,12.146341,4693,1,L,4,F,X,X,X,A


In [ ]:
profile = ProfileReport(donors)

# saving the report as an output.html file

profile.to_file("output_all.html")


# Data Preparation

## Missing Values

In [63]:
donors.dtypes

ODATEDW     object
OSOURCE     object
TCODE        int64
STATE       object
ZIP         object
             ...  
RFA_2A      object
MDMAUD_R    object
MDMAUD_F    object
MDMAUD_A    object
GEOCODE2    object
Length: 475, dtype: object

In [64]:
donors["ODATEDW"] = pd.to_datetime(donors["ODATEDW"])
donors["ODATEDW"] = donors["ODATEDW"].apply(lambda x: x.strftime("%m-%Y")) #In case we want to know the name's month, in spite of m, we write B
donors

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,01-2009,GRI,0,IL,61081,,,1957-12-01,0,,...,7.741935,95515,0,L,4,E,X,X,X,C
1,01-2014,BOA,1,CA,91326,,,1972-02-01,0,,...,15.666667,148535,0,L,2,G,X,X,X,A
2,01-2010,AMH,1,NC,27017,,,NaN,0,,...,7.481481,15078,1,L,4,E,X,X,X,C
3,01-2007,BRY,0,CA,95953,,,1948-01-01,0,,...,6.812500,172556,1,L,4,E,X,X,X,C
4,01-2006,,0,FL,33176,,,1940-01-01,0,X,...,6.864865,7112,1,L,2,F,X,X,X,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,01-2016,ASE,1,AK,99504,,,NaN,0,,...,25.000000,184568,0,L,1,G,X,X,X,C
95408,01-2016,DCD,1,TX,77379,,,1970-01-01,0,,...,20.000000,122706,1,L,1,F,X,X,X,A
95409,01-2015,MBC,1,MI,48910,,,1958-01-01,0,,...,8.285714,189641,1,L,3,E,X,X,X,B
95410,01-2006,PRV,0,CA,91320,,,1960-05-01,0,X,...,12.146341,4693,1,L,4,F,X,X,X,A


In [65]:
donors.dtypes

ODATEDW     object
OSOURCE     object
TCODE        int64
STATE       object
ZIP         object
             ...  
RFA_2A      object
MDMAUD_R    object
MDMAUD_F    object
MDMAUD_A    object
GEOCODE2    object
Length: 475, dtype: object

In [66]:
donors.replace(" ", np.nan, inplace=True #NÃO SE PODE

In [67]:
donors.replace("", np.nan, inplace=True)

In [68]:
donors.isna().sum()

ODATEDW       0
OSOURCE     928
TCODE         0
STATE         0
ZIP           0
           ... 
RFA_2A        0
MDMAUD_R      0
MDMAUD_F      0
MDMAUD_A      0
GEOCODE2    319
Length: 475, dtype: int64

In [69]:
df = donors.loc[:, donors.isin([np.nan]).mean() < 0.6]

In [70]:
#df = donors.loc[:, donors.isin([np.nan]).mean() < 0.6] - It's the same as below
donors.dropna(axis=1, how="all", thresh=(donors.shape[0]*0.4), inplace=True) #we are removing all the columns that have more than 60% of Missing Values, and the thresh means 'Require that many non-NA values'

In [71]:
donors

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,DOB,NOEXCH,MDMAUD,DOMAIN,HOMEOWNR,...,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,01-2009,GRI,0,IL,61081,1957-12-01,0,XXXX,T2,NaN,...,7.741935,95515,0,L,4,E,X,X,X,C
1,01-2014,BOA,1,CA,91326,1972-02-01,0,XXXX,S1,H,...,15.666667,148535,0,L,2,G,X,X,X,A
2,01-2010,AMH,1,NC,27017,NaN,0,XXXX,R2,U,...,7.481481,15078,1,L,4,E,X,X,X,C
3,01-2007,BRY,0,CA,95953,1948-01-01,0,XXXX,R2,U,...,6.812500,172556,1,L,4,E,X,X,X,C
4,01-2006,NaN,0,FL,33176,1940-01-01,0,XXXX,S2,H,...,6.864865,7112,1,L,2,F,X,X,X,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,01-2016,ASE,1,AK,99504,NaN,0,XXXX,C2,NaN,...,25.000000,184568,0,L,1,G,X,X,X,C
95408,01-2016,DCD,1,TX,77379,1970-01-01,0,XXXX,C1,H,...,20.000000,122706,1,L,1,F,X,X,X,A
95409,01-2015,MBC,1,MI,48910,1958-01-01,0,XXXX,C3,NaN,...,8.285714,189641,1,L,3,E,X,X,X,B
95410,01-2006,PRV,0,CA,91320,1960-05-01,0,XXXX,C1,H,...,12.146341,4693,1,L,4,F,X,X,X,A


In [72]:
donors.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
ODATEDW,95412,54,01-2015,15358,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OSOURCE,94484,895,MBC,4539,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCODE,95412,NaN,NaN,NaN,54.2231,953.844,0,0,1,2,72002
STATE,95412,57,CA,17343,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZIP,95412,19938,85351,61,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
RFA_2A,95412,4,F,46964,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MDMAUD_R,95412,5,X,95118,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MDMAUD_F,95412,4,X,95118,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MDMAUD_A,95412,5,X,95118,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
#We know that the variables are categoricals, when it has a value in Unique, Top and Freq. So, let's first take all the metric_features to a list, as this ones are from the describe only

metric_features = list(donors.describe().columns)
metric_features

['TCODE',
 'INCOME',
 'WEALTH1',
 'HIT',
 'MBCRAFT',
 'MBGARDEN',
 'MBBOOKS',
 'MBCOLECT',
 'MAGFAML',
 'MAGFEM',
 'MAGMALE',
 'PUBGARDN',
 'PUBCULIN',
 'PUBHLTH',
 'PUBDOITY',
 'PUBNEWFN',
 'PUBPHOTO',
 'PUBOPP',
 'MALEMILI',
 'MALEVET',
 'VIETVETS',
 'WWIIVETS',
 'LOCALGOV',
 'STATEGOV',
 'FEDGOV',
 'WEALTH2',
 'POP901',
 'POP902',
 'POP903',
 'POP90C1',
 'POP90C2',
 'POP90C3',
 'POP90C4',
 'POP90C5',
 'ETH1',
 'ETH2',
 'ETH3',
 'ETH4',
 'ETH5',
 'ETH6',
 'ETH7',
 'ETH8',
 'ETH9',
 'ETH10',
 'ETH11',
 'ETH12',
 'ETH13',
 'ETH14',
 'ETH15',
 'ETH16',
 'AGE901',
 'AGE902',
 'AGE903',
 'AGE904',
 'AGE905',
 'AGE906',
 'AGE907',
 'CHIL1',
 'CHIL2',
 'CHIL3',
 'AGEC1',
 'AGEC2',
 'AGEC3',
 'AGEC4',
 'AGEC5',
 'AGEC6',
 'AGEC7',
 'CHILC1',
 'CHILC2',
 'CHILC3',
 'CHILC4',
 'CHILC5',
 'HHAGE1',
 'HHAGE2',
 'HHAGE3',
 'HHN1',
 'HHN2',
 'HHN3',
 'HHN4',
 'HHN5',
 'HHN6',
 'MARR1',
 'MARR2',
 'MARR3',
 'MARR4',
 'HHP1',
 'HHP2',
 'DW1',
 'DW2',
 'DW3',
 'DW4',
 'DW5',
 'DW6',
 'DW7',
 'DW8',
 

In [74]:
donors["INCOME"].unique()

array([nan,  6.,  3.,  1.,  4.,  2.,  7.,  5.])

In [75]:
donors["MSA"].unique()

array([   0., 4480., 9340., 5000., 2030., 3960., 5360., 3480.,  760.,
       2160., 5960., 5120.,  440., 6580., 1600.,  680., 3360., 7040.,
       2920., 3120., 6440.,  640., 6120., 3000., 7600., 2680., 5080.,
       1360., 2180., 4420., 1960., 6640., 7360., 5945., 1920., 1145.,
       7520., 2800., 1520., 7510., 1620., 4520., 7320.,  720., 4720.,
       1560., 8960.,   40., 6720.,  520., 9360., 3710., 8280., 6760.,
       3160., 5560., 3600., 6780., 5800., 4120., 5160., 5880.,  380.,
       3440., 2710., 2020., 9040., 6200., 6880., 6920., 2760., 7080.,
       4080., 2580., 4100., 5790., 4200.,  600., 4680., 4040., 1640.,
       5775., 8720., 3980., 2240., 1720., 5910., 9260.,  200., 7400.,
       1020., 3290., 2720., 7240., 3800., 8120.,  870., 2320., 3760.,
       1040., 2840., 3880.,  920., 6080.,  460.,  320., 1400., 3320.,
       2640., 8200., 4890., 3660., 2670., 1000., 2520., 2440., 8520.,
       3720., 2080., 7500., 3920., 1125., 4400.,  840., 7485., 8750.,
       8735., 1150.,

In [76]:
donors["HPHONE_D"].unique()

array([0, 1], dtype=int64)

In [77]:
"HPHONE_D" in metric_features

True

In [78]:
"PEPSTRFL" in non_metric_features

True

In [79]:
#Taking all the non_metrics_features

non_metric_features = donors.columns.drop(metric_features).to_list()
non_metric_features

['ODATEDW',
 'OSOURCE',
 'STATE',
 'ZIP',
 'DOB',
 'NOEXCH',
 'MDMAUD',
 'DOMAIN',
 'HOMEOWNR',
 'GENDER',
 'DATASRCE',
 'LIFESRC',
 'PEPSTRFL',
 'ADATE_2',
 'ADATE_3',
 'ADATE_4',
 'ADATE_5',
 'ADATE_6',
 'ADATE_7',
 'ADATE_8',
 'ADATE_9',
 'ADATE_10',
 'ADATE_11',
 'ADATE_12',
 'ADATE_13',
 'ADATE_14',
 'ADATE_16',
 'ADATE_17',
 'ADATE_18',
 'ADATE_19',
 'ADATE_20',
 'ADATE_21',
 'ADATE_22',
 'ADATE_23',
 'ADATE_24',
 'RFA_2',
 'RFA_3',
 'RFA_4',
 'RFA_5',
 'RFA_6',
 'RFA_7',
 'RFA_8',
 'RFA_9',
 'RFA_10',
 'RFA_11',
 'RFA_12',
 'RFA_13',
 'RFA_14',
 'RFA_16',
 'RFA_17',
 'RFA_18',
 'RFA_19',
 'RFA_20',
 'RFA_21',
 'RFA_22',
 'RFA_23',
 'RFA_24',
 'MAXADATE',
 'MINRDATE',
 'MAXRDATE',
 'LASTDATE',
 'FISTDATE',
 'NEXTDATE',
 'RFA_2R',
 'RFA_2A',
 'MDMAUD_R',
 'MDMAUD_F',
 'MDMAUD_A',
 'GEOCODE2']

# FIXING THE CATHEGORICALS 

In [80]:
donors["RFA_2F"] = donors["RFA_2F"].astype(str)

In [81]:
donors["HPHONE_D"] = donors["HPHONE_D"].astype(str)

In [82]:
#donors["MSA"].apply(lambda x: int(float(x)) if (x!=np.nan) else (np.nan))
#donors["MSA"].apply(lambda x: round(x) if (x!=np.nan) else (np.nan))
donors["MSA"] = donors["MSA"].astype(str)
donors["MSA"] = donors["MSA"].apply(lambda x: x[:-2])
donors["MSA"]

0           0
1        4480
2           0
3        9340
4        5000
         ... 
95407     380
95408    3360
95409    4040
95410    8735
95411    9200
Name: MSA, Length: 95412, dtype: object

In [83]:
donors["WEALTH1"] = donors["WEALTH1"].astype(str)

In [84]:
donors["WEALTH2"] = donors["WEALTH2"].astype(str)

In [85]:
donors["INCOME"] = donors["INCOME"].astype(str)

In [86]:
donors["TCODE"] = donors["TCODE"].astype(str)

In [87]:
#Updating the Non_Metric_Features and Metric_Features
non_metric_features.append(["RFA_2F","HPHONE_D","MSA","WEALTH1","WEALTH2","INCOME","TCODE"])

In [88]:
to_remove = {"RFA_2F","HPHONE_D","MSA","WEALTH1","WEALTH2","INCOME","TCODE"}
metric_features = [x for x in metric_features if x not in to_remove]
metric_features

['HIT',
 'MBCRAFT',
 'MBGARDEN',
 'MBBOOKS',
 'MBCOLECT',
 'MAGFAML',
 'MAGFEM',
 'MAGMALE',
 'PUBGARDN',
 'PUBCULIN',
 'PUBHLTH',
 'PUBDOITY',
 'PUBNEWFN',
 'PUBPHOTO',
 'PUBOPP',
 'MALEMILI',
 'MALEVET',
 'VIETVETS',
 'WWIIVETS',
 'LOCALGOV',
 'STATEGOV',
 'FEDGOV',
 'POP901',
 'POP902',
 'POP903',
 'POP90C1',
 'POP90C2',
 'POP90C3',
 'POP90C4',
 'POP90C5',
 'ETH1',
 'ETH2',
 'ETH3',
 'ETH4',
 'ETH5',
 'ETH6',
 'ETH7',
 'ETH8',
 'ETH9',
 'ETH10',
 'ETH11',
 'ETH12',
 'ETH13',
 'ETH14',
 'ETH15',
 'ETH16',
 'AGE901',
 'AGE902',
 'AGE903',
 'AGE904',
 'AGE905',
 'AGE906',
 'AGE907',
 'CHIL1',
 'CHIL2',
 'CHIL3',
 'AGEC1',
 'AGEC2',
 'AGEC3',
 'AGEC4',
 'AGEC5',
 'AGEC6',
 'AGEC7',
 'CHILC1',
 'CHILC2',
 'CHILC3',
 'CHILC4',
 'CHILC5',
 'HHAGE1',
 'HHAGE2',
 'HHAGE3',
 'HHN1',
 'HHN2',
 'HHN3',
 'HHN4',
 'HHN5',
 'HHN6',
 'MARR1',
 'MARR2',
 'MARR3',
 'MARR4',
 'HHP1',
 'HHP2',
 'DW1',
 'DW2',
 'DW3',
 'DW4',
 'DW5',
 'DW6',
 'DW7',
 'DW8',
 'DW9',
 'HV1',
 'HV2',
 'HV3',
 'HV4',
 'HU1'

## Correlation

Features with high correlation are more linearly dependent and hence have almost the same effect on the dependent variable. So, when two features have high correlation, we can drop one of the two features. Basically, when features that have a correlation higher than 0.9, approximately.

This correlations can only be made between metric_features

In [89]:
donors[metric_features].corr(method="pearson")

,HIT,MBCRAFT,MBGARDEN,MBBOOKS,MBCOLECT,MAGFAML,MAGFEM,MAGMALE,PUBGARDN,PUBCULIN,...,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN
HIT,1.000000,0.273154,0.190401,0.516363,0.280819,0.374718,0.256148,0.164642,0.304608,0.347600,...,0.045792,0.033941,0.043545,0.044594,-0.029573,-0.002322,-0.007213,0.008685,-0.014705,0.000420
MBCRAFT,0.273154,1.000000,0.213852,0.284780,0.113132,0.268072,0.171070,0.075541,0.153036,0.172247,...,0.005588,0.000309,0.011206,0.015940,-0.019897,-0.005536,-0.011943,0.001517,-0.016241,-0.001785
MBGARDEN,0.190401,0.213852,1.000000,0.200152,0.069470,0.186100,0.077892,0.031956,0.207128,0.142152,...,0.005571,0.003394,0.026228,0.030418,-0.025969,-0.014832,-0.018356,0.002419,-0.027719,-0.017925
MBBOOKS,0.516363,0.284780,0.200152,1.000000,0.157116,0.367868,0.296526,0.151781,0.507233,0.586836,...,0.012566,0.010838,-0.009686,-0.016488,0.002255,0.015219,0.012921,0.006022,0.015487,0.005684
MBCOLECT,0.280819,0.113132,0.069470,0.157116,1.000000,0.161029,0.122873,0.082422,0.077197,0.132772,...,0.035829,0.036442,0.018055,0.016457,0.005579,0.026214,0.026024,0.004416,0.026210,0.003236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MAXRAMNT,-0.002322,-0.005536,-0.014832,0.015219,0.026214,0.000703,0.008280,0.013166,0.002663,0.009511,...,0.142544,0.557428,-0.043655,-0.067850,0.293249,1.000000,0.562620,0.032012,0.766810,-0.011947
LASTGIFT,-0.007213,-0.011943,-0.018356,0.012921,0.026024,0.002448,0.009117,0.019529,0.002778,0.009107,...,0.097370,0.323751,-0.181368,-0.190158,0.531548,0.562620,1.000000,0.073174,0.784187,-0.000427
TIMELAG,0.008685,0.001517,0.002419,0.006022,0.004416,0.005385,0.007897,0.011591,0.003912,0.010509,...,-0.029677,-0.032924,-0.110699,-0.101840,0.073466,0.032012,0.073174,1.000000,0.077111,0.019095
AVGGIFT,-0.014705,-0.016241,-0.027719,0.015487,0.026210,-0.001544,0.004539,0.027879,0.000354,0.005725,...,0.088357,0.365560,-0.256992,-0.268607,0.754563,0.766810,0.784187,0.077111,1.000000,0.011507


In [90]:
corr_serie = donors[metric_features].corr(method="pearson").unstack().sort_values().drop_duplicates() #Create a serie with all the correlations existed.

In [91]:
top_corr = corr_serie[(corr_serie>=0.85) & (corr_serie!=1) | (corr_serie<=-0.85)] #Knowing which are the highest corr between features to eliminate after.

In [92]:
top_corr

HC18    HC17     -0.935513
HC7     HC8      -0.933972
HU2     HU1      -0.907210
HVP5    HVP4      0.850590
IC8     IC17      0.850899
                    ...   
POP901  POP903    0.981225
        POP902    0.988491
ETHC5   ETH2      0.989670
HV2     HV1       0.993412
HHAGE1  HHAGE3    0.993997
Length: 143, dtype: float64

# Data Normalization

### Min-Max

In [94]:
donors_minmax= donors.copy()
scaler = MinMaxScaler()
scaled_feat = scaler.fit_transform(donors_minmax[metric_features])
scaled_feat

array([[0.        ,        nan,        nan, ..., 0.00367647, 0.00646453,
        0.49804461],
       [0.06639004, 0.        , 0.        , ..., 0.01654412, 0.01439947,
        0.77451011],
       [0.00829876, 0.        , 0.        , ..., 0.01102941, 0.00620374,
        0.07861694],
       ...,
       [0.        ,        nan,        nan, ..., 0.00275735, 0.00700901,
        0.98885169],
       [0.        ,        nan,        nan, ..., 0.00367647, 0.01087461,
        0.02446579],
       [0.01244813, 0.        , 0.        , ..., 0.00551471, 0.09563211,
        0.96524627]])

In [95]:
donors_minmax[metric_features] = scaled_feat
donors_minmax.head()

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,DOB,NOEXCH,MDMAUD,DOMAIN,HOMEOWNR,...,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,01-2009,GRI,0,IL,61081,1957-12-01,0,XXXX,T2,NaN,...,0.006465,0.498045,0,L,4,E,X,X,X,C
1,01-2014,BOA,1,CA,91326,1972-02-01,0,XXXX,S1,H,...,0.014399,0.774510,0,L,2,G,X,X,X,A
2,01-2010,AMH,1,NC,27017,NaN,0,XXXX,R2,U,...,0.006204,0.078617,1,L,4,E,X,X,X,C
3,01-2007,BRY,0,CA,95953,1948-01-01,0,XXXX,R2,U,...,0.005534,0.899764,1,L,4,E,X,X,X,C
4,01-2006,NaN,0,FL,33176,1940-01-01,0,XXXX,S2,H,...,0.005586,0.037079,1,L,2,F,X,X,X,A


In [96]:
donors_minmax[metric_features].describe().round(2)

,HIT,MBCRAFT,MBGARDEN,MBBOOKS,MBCOLECT,MAGFAML,MAGFEM,MAGMALE,PUBGARDN,PUBCULIN,...,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN
count,95412.00,42558.00,42558.00,42558.00,42498.00,42558.00,42558.00,42558.00,42558.00,42558.00,...,95412.00,95412.00,95412.00,95412.00,95412.00,95412.00,95412.00,85439.00,95412.00,95412.00
mean,0.01,0.03,0.01,0.12,0.01,0.05,0.03,0.02,0.03,0.02,...,0.15,0.01,0.04,0.12,0.01,0.00,0.02,0.01,0.01,0.50
std,0.04,0.08,0.07,0.19,0.05,0.09,0.08,0.07,0.10,0.07,...,0.06,0.01,0.04,0.11,0.01,0.01,0.01,0.01,0.01,0.29
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.13,0.00,0.01,0.05,0.00,0.00,0.01,0.00,0.01,0.25
50%,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.14,0.01,0.03,0.10,0.00,0.00,0.02,0.01,0.01,0.50
75%,0.01,0.00,0.00,0.22,0.00,0.11,0.00,0.00,0.00,0.00,...,0.16,0.01,0.05,0.17,0.01,0.00,0.02,0.01,0.01,0.75
max,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


### Z-Score

In [98]:
donors_standard = donors.copy()

In [99]:
scaler = StandardScaler()
scaled_feat = scaler.fit_transform(donors_standard[metric_features])
scaled_feat

array([[-3.56880959e-01,             nan,             nan, ...,
        -4.98434536e-01, -5.20508933e-01, -4.76042798e-03],
       [ 1.36228288e+00, -3.23551227e-01, -2.25761190e-01, ...,
         1.20613990e+00,  2.15310435e-01,  9.54282083e-01],
       [-1.41985479e-01, -3.23551227e-01, -2.25761190e-01, ...,
         4.75607997e-01, -5.44692352e-01, -1.45973030e+00],
       ...,
       [-3.56880959e-01,             nan,             nan, ...,
        -6.20189853e-01, -4.70018517e-01,  1.69782040e+00],
       [-3.56880959e-01,             nan,             nan, ...,
        -4.98434536e-01, -1.11555348e-01, -1.64757746e+00],
       [-3.45377388e-02, -3.23551227e-01, -2.25761190e-01, ...,
        -2.54923903e-01,  7.74814698e+00,  1.61593459e+00]])

In [100]:
donors_standard[metric_features] = scaled_feat
donors_standard.head()

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,DOB,NOEXCH,MDMAUD,DOMAIN,HOMEOWNR,...,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,01-2009,GRI,0,IL,61081,1957-12-01,0,XXXX,T2,NaN,...,-0.520509,-0.004760,0,L,4,E,X,X,X,C
1,01-2014,BOA,1,CA,91326,1972-02-01,0,XXXX,S1,H,...,0.215310,0.954282,0,L,2,G,X,X,X,A
2,01-2010,AMH,1,NC,27017,NaN,0,XXXX,R2,U,...,-0.544692,-1.459730,1,L,4,E,X,X,X,C
3,01-2007,BRY,0,CA,95953,1948-01-01,0,XXXX,R2,U,...,-0.606808,1.388782,1,L,4,E,X,X,X,C
4,01-2006,NaN,0,FL,33176,1940-01-01,0,XXXX,S2,H,...,-0.601946,-1.603822,1,L,2,F,X,X,X,A


In [101]:
donors_standard[metric_features].describe().round(2)

,HIT,MBCRAFT,MBGARDEN,MBBOOKS,MBCOLECT,MAGFAML,MAGFEM,MAGMALE,PUBGARDN,PUBCULIN,...,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN
count,95412.00,42558.00,42558.00,42558.00,42498.00,42558.00,42558.00,42558.00,42558.00,42558.00,...,95412.00,95412.00,95412.00,95412.00,95412.00,95412.00,95412.00,85439.00,95412.00,95412.00
mean,0.00,0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,...,-0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,0.00
std,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
min,-0.36,-0.32,-0.23,-0.67,-0.22,-0.55,-0.33,-0.24,-0.29,-0.34,...,-2.61,-0.77,-1.01,-1.12,-0.90,-0.60,-1.24,-0.99,-1.12,-1.73
25%,-0.36,-0.32,-0.23,-0.67,-0.22,-0.55,-0.33,-0.24,-0.29,-0.34,...,-0.41,-0.54,-0.77,-0.68,-0.56,-0.24,-0.52,-0.50,-0.46,-0.87
50%,-0.36,-0.32,-0.23,-0.67,-0.22,-0.55,-0.33,-0.24,-0.29,-0.34,...,-0.19,-0.22,-0.30,-0.23,-0.33,-0.12,-0.17,-0.25,-0.16,-0.00
75%,-0.03,-0.32,-0.23,0.53,-0.22,0.67,-0.33,-0.24,-0.29,-0.34,...,0.03,0.22,0.40,0.43,0.24,0.12,0.19,0.35,0.20,0.87
max,25.54,12.44,15.04,4.72,19.96,10.42,12.79,14.23,9.91,13.69,...,14.34,79.11,26.58,7.94,112.94,198.66,70.41,131.48,91.61,1.74


In [97]:
sns.set()

fig, axes = plt.subplots(2, ceil(len(non_metric_features) / 2), figsize=(30, 11))

for ax, feat in zip(axes.flatten(), non_metric_features):
    sns.countplot(x=donors[feat], ax=ax, color='#007acc')

title = "Categorical/Low Cardinality Variables' Absolute Frequencies"
plt.suptitle(title)

plt.show()

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

Error in callback <function flush_figures at 0x0000019DE848FB80> (for post_execute):


KeyboardInterrupt: 

In [43]:
q25 = donors.quantile(.25)
q75 = donors.quantile(.75)
iqr = (q75 - q25)

upper_lim = q75 + 3 * iqr
lower_lim = q25 - 3 * iqr

filters2 = []
for metric in metric_features:
    llim = lower_lim[metric]
    ulim = upper_lim[metric]
    filters2.append(donors[metric].between(llim, ulim, inclusive=True))

filters2 = pd.Series(np.all(filters2, 0))
df_2 = donors[filters2]
print('Percentage of data kept after removing outliers:', np.round(df_2.shape[0] / donors.shape[0], 4))

Percentage of data kept after removing outliers: 0.0025


In [44]:
df_2

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,DOB,NOEXCH,MDMAUD,DOMAIN,HOMEOWNR,...,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
117,2015-01-01,WRG,0,OK,73159,1952-01-01,0,XXXX,C1,H,...,8.333333,113495,1,L,3,E,X,X,X,B
403,2010-01-01,AML,0,AL,35811,2000-01-01,0,XXXX,C2,H,...,3.812500,48286,1,L,4,D,X,X,X,B
1244,2006-01-01,ASC,1,TX,78745,1936-12-01,0,XXXX,S1,H,...,12.687500,127287,1,L,3,F,X,X,X,B
1459,2006-01-01,DRK,1,WA,99016,1951-07-01,0,XXXX,T2,H,...,8.370370,183744,1,L,2,E,X,X,X,B
1606,2015-01-01,CWR,28,AL,35068,1944-01-01,0,XXXX,T2,H,...,13.000000,46856,1,L,1,F,X,X,X,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93718,2009-01-01,MCC,0,WA,98038,1976-01-01,0,XXXX,T1,H,...,6.952381,178939,0,L,1,E,X,X,X,A
94022,2009-01-01,GHS,0,GA,30135,1984-12-01,0,XXXX,T2,H,...,6.950000,26795,0,L,4,D,X,X,X,A
94104,2006-01-01,BHG,2,SC,29801,1946-08-01,0,XXXX,T2,H,...,6.333333,25269,1,L,1,E,X,X,X,B
94255,2013-01-01,AGR,2,IN,46737,NaN,0,XXXX,R2,U,...,6.333333,62068,1,L,3,E,X,X,X,D


In [ ]:
import matplotlib.pyplot as plt
from sklearn.neighbors import LocalOutlierFactor


np.random.seed(42)

ground_truth = np.ones(len(X), dtype=int)
ground_truth[-n_outliers:] = -1

# fit the model for outlier detection (default)
clf = LocalOutlierFactor(n_neighbors=20, contamination=0.1)
# use fit_predict to compute the predicted labels of the training samples
# (when LOF is used for outlier detection, the estimator has no predict,
# decision_function and score_samples methods).
y_pred = clf.fit_predict(X)
n_errors = (y_pred != ground_truth).sum()
X_scores = clf.negative_outlier_factor_

plt.title("Local Outlier Factor (LOF)")
plt.scatter(X[:, 0], X[:, 1], color='k', s=3., label='Data points')
# plot circles with radius proportional to the outlier scores
radius = (X_scores.max() - X_scores) / (X_scores.max() - X_scores.min())
plt.scatter(X[:, 0], X[:, 1], s=1000 * radius, edgecolors='r',
            facecolors='none', label='Outlier scores')
plt.axis('tight')
plt.xlim((-5, 5))
plt.ylim((-5, 5))
plt.xlabel("prediction errors: %d" % (n_errors))
legend = plt.legend(loc='upper left')
legend.legendHandles[0]._sizes = [10]
legend.legendHandles[1]._sizes = [20]
plt.show()

In [ ]:
df_2

In [ ]:
fig = plt.figure(figsize=(320, 320))

corr = np.round(donors[metric_features].corr(method="pearson"), decimals=2)

mask_annot = np.absolute(corr.values) >= 0.5
annot = np.where(mask_annot, corr.values, np.full(corr.shape,""))

sns.heatmap(data=corr, annot=annot, cmap=sns.diverging_palette(220, 10, as_cmap=True), 
            fmt='s', vmin=-1, vmax=1, center=0, square=True, linewidths=.5)

# Layout
fig.subplots_adjust(top=0.95)
fig.suptitle("Correlation Matrix", fontsize=20)

plt.savefig(os.path.join('correlation_matrix.png'), dpi=200)

plt.show()

In [45]:
donors[non_metric_features].corr(method="spearman")

TypeError: unhashable type: 'list'

In [34]:
donors.drop("WEALTH1",axis=1,inplace=True) #Corr=0.869

#After some time comparing each correlation value, I thought this wasn't the best efficient approach. So, below, there is another approach for the problem, evaluating the coeficient of correlation, as this allows us to check which are the highest correlations and the lowest too.

In [35]:
corr_serie = donors.corr(method="pearson").unstack().sort_values().drop_duplicates() #Create a serie with all the correlations existed.

In [36]:
corr_serie.shape[0]

48517

In [37]:
top_corr = corr_serie[(corr_serie>=0.85) & (corr_serie!=1) | (corr_serie<=-0.85)] #Knowing which are the highest corr between features to eliminate after.

First, I have looked if the variable I was eliminating was the one with less missing values and also concerning its meaning.

In [38]:
top_corr[:50]

HC17    HC18       -0.935513
HC8     HC7        -0.933972
HU1     HU2        -0.907210
HVP4    HVP5        0.850590
IC8     IC17        0.850899
HHAGE1  AGE904      0.852352
ETH13   LSC2        0.852576
HHAS1   AGEC6       0.853758
AGE907  SEC4        0.853773
        HHN3        0.854692
DW4     HUPA2       0.855566
OEDC2   STATEGOV    0.856320
HUPA1   HUPA5       0.856986
HHAGE3  HHAGE2      0.857450
CHILC1  CHIL1       0.861792
HHAGE2  AGEC7       0.861802
HHD2    HHP2        0.863814
ETHC3   AGE901      0.866007
HC20    HC21        0.866811
HHD1    RHP3        0.869850
ETH4    LSC3        0.870757
HVP1    HVP6        0.872224
HHAS4   IC15        0.873561
IC2     IC5         0.873939
HHD4    HHP1        0.874815
HC2     HC8         0.876162
HC7     HC6         0.876531
ETHC3   AGEC6       0.880242
IC16    IC7         0.881669
IC5     IC4         0.882082
IC18    IC9         0.883419
HC5     HC4         0.884207
RP4     RP3         0.885588
RP2     HV4         0.885616
HHN5    HHN4  

In [39]:
donors.loc[:,["IC2","IC5"]]

,IC2,IC5
0,318,12883
1,1096,36175
2,292,11576
3,388,15130
4,250,9836
...,...,...
95407,481,18807
95408,836,26538
95409,264,12178
95410,544,15948


In [40]:
top_corr[50:100]

HHAGE3    AGEC6       0.899803
HHN4      RHP3        0.899944
HVP6      HV2         0.902442
HUPA2     DW5         0.905181
HHD1      HHP1        0.905384
AGEC6     HHAGE1      0.905412
HVP6      HV1         0.906695
HHD3      HHD2        0.907693
HV4       RP1         0.908495
HHN3      HHD4        0.908945
HHN1      HHD11       0.909182
ETH13     ETH5        0.909272
HHN5      HHN6        0.910884
HV2       HVP2        0.911005
TPE4      TPE3        0.911778
HHD4      HHD1        0.913323
MARR3     AGEC7       0.913703
HUR2      RHP1        0.914362
HVP4      HVP3        0.914423
CARDGIFT  NGIFTALL    0.914586
HHN3      RHP3        0.915850
LFC4      LFC1        0.917379
RP2       RP1         0.919733
LFC5      LFC1        0.921067
HHAS1     HHAGE3      0.921169
          HHAGE1      0.921519
HHD2      HHD5        0.922107
AGE904    AGE906      0.922192
AGE906    AGE902      0.922731
RHP2      HUR2        0.924274
HV1       HVP1        0.924447
MARR1     HHD3        0.925934
DW4     

A Mexican person can also be referred to as a Hispanic however not all Hispanics are Mexicans, so be very careful!

In [41]:
top_corr[100:144]

HHN3      HHP2       0.948080
CARDPROM  NUMPROM    0.949052
IC12      IC21       0.949089
IC10      IC19       0.949650
AFC5      AFC4       0.950246
IC1       IC2        0.951252
DW1       DW2        0.951974
HHN4      HHP1       0.952375
HHN3      HHD1       0.953158
ETH2      ETHC4      0.953195
IC3       IC2        0.955143
AGE901    AGE904     0.955908
HHP1      HHN3       0.956023
DW6       HUPA2      0.956113
IC11      IC20       0.957710
EIC1      OCC9       0.958144
ETH5      LSC2       0.958943
AGE905    AGE903     0.960033
HHP2      RHP3       0.962883
AGE905    AGE902     0.964050
RHP3      HHP1       0.965752
HHN4      HHN3       0.965966
HHP1      HHP2       0.966611
AGE903    AGE902     0.966649
AGE906    AGE903     0.966688
IC14      IC23       0.968103
IC4       IC3        0.971395
RHP2      RHP1       0.971449
LFC4      LFC2       0.971605
DW6       DW5        0.971715
IC2       IC4        0.973616
DW5       DW4        0.974774
IC3       IC1        0.975308
AGE905    

In [42]:
donors.drop(["HC18","HC8","HV2","HV4","HVP2","HVP4","HVP6","IC1","IC3","IC4","IC6","IC7","IC8","IC9","IC10","IC11","IC12","IC13","IC14","AGE901","AGE903","AGE904","AGE905","AGE906","LSC1","LSC2","LSC3","LSC4","HHAGE1","HHAGE3","DW2","DW4","DW5","DW6","HHD1","HHD2","HHD3","HHD4","HHD6","HHD9","HHD11","HHN1","HHN4","HHN5","HHN6","ETH13","ETH14","ETH15","ETH16","TPE4","TPE5","TPE6","MARR3","HUR1","HUR2","CARDGIFT","CARDPROM","LFC1","LFC2","LFC3","LFC5","RP1","RP3","ETHC2","ETHC5","OEDC1","OEDC2","OEDC3","CHILC1","CHILC2","CHILC3","CHILC4","CHILC5","CHIL1","CHIL2","CHIL3","HC4","HC6","HC8","HHAS4","HHP1","HHP2","AFC3","AFC6","OCC9","POP903"],axis=1,inplace=True)
donors

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,DOB,NOEXCH,MDMAUD,DOMAIN,HOMEOWNR,...,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,01-2009,GRI,0,IL,61081,1957-12-01,0,XXXX,T2,NaN,...,7.741935,95515,0,L,4,E,X,X,X,C
1,01-2014,BOA,1,CA,91326,1972-02-01,0,XXXX,S1,H,...,15.666667,148535,0,L,2,G,X,X,X,A
2,01-2010,AMH,1,NC,27017,NaN,0,XXXX,R2,U,...,7.481481,15078,1,L,4,E,X,X,X,C
3,01-2007,BRY,0,CA,95953,1948-01-01,0,XXXX,R2,U,...,6.812500,172556,1,L,4,E,X,X,X,C
4,01-2006,NaN,0,FL,33176,1940-01-01,0,XXXX,S2,H,...,6.864865,7112,1,L,2,F,X,X,X,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,01-2016,ASE,1,AK,99504,NaN,0,XXXX,C2,NaN,...,25.000000,184568,0,L,1,G,X,X,X,C
95408,01-2016,DCD,1,TX,77379,1970-01-01,0,XXXX,C1,H,...,20.000000,122706,1,L,1,F,X,X,X,A
95409,01-2015,MBC,1,MI,48910,1958-01-01,0,XXXX,C3,NaN,...,8.285714,189641,1,L,3,E,X,X,X,B
95410,01-2006,PRV,0,CA,91320,1960-05-01,0,XXXX,C1,H,...,12.146341,4693,1,L,4,F,X,X,X,A


In [43]:
#Let's see with the new table

In [44]:
corr_serie_2 = donors.corr(method="pearson").unstack().sort_values().drop_duplicates() #Create a serie with all the correlations existed.

In [45]:
top_corr_2 = corr_serie_2[(corr_serie_2>=0.85) & (corr_serie_2!=1) | (corr_serie_2<=-0.85)] #Knowing which are the highest corr between features to eliminate after.

In [46]:
top_corr_2 #As minhas dúvidas

HU2     HU1      -0.907210
AGEC6   HHAS1     0.853758
SEC4    AGE907    0.853773
AGE907  HHN3      0.854692
HUPA1   HUPA5     0.856986
HHAGE2  AGEC7     0.861802
HC20    HC21      0.866811
IC2     IC5       0.873939
AGEC6   ETHC3     0.880242
HUPA2   HUPA6     0.891038
HHN3    RHP3      0.915850
HVP1    HV1       0.924447
AFC5    AFC4      0.950246
ETH2    ETHC4     0.953195
RHP1    RHP2      0.971449
AFC2    AFC1      0.979608
POP902  POP901    0.988491
dtype: float64

In [53]:
no_corr = corr_serie_2[(corr_serie_2>=-0.0001) & (corr_serie_2<=0.0001)] #The Lowest Correlations -> Because we don't want variables with zero correlation

In [54]:
no_corr

RFA_2F    TPE8       -0.000099
TIMELAG   HHD12      -0.000094
ETH10     MARR2      -0.000091
OCC6      CARDPM12   -0.000090
STATEGOV  ETH8       -0.000078
OCC6      TIMELAG    -0.000074
TIMELAG   EIC16      -0.000071
AC2       EC8        -0.000068
EIC4      CARDPM12   -0.000067
POP90C1   CARDPM12   -0.000059
TCODE     POP90C4    -0.000058
MARR4     MALEMILI   -0.000039
LOCALGOV  ANC8       -0.000036
VOC3      AC2        -0.000033
TCODE     HHN3       -0.000029
          OCC3       -0.000027
          EC6        -0.000027
VOC1      ETH12      -0.000021
TCODE     LASTGIFT   -0.000010
OEDC4     ETH10      -0.000009
VC1       ANC4       -0.000004
HUPA1     EIC14      -0.000002
MAXRAMNT  ANC1        0.000005
ANC13     ANC1        0.000022
EIC5      DMA         0.000028
AGEC7     TIMELAG     0.000029
TCODE     AGEC2       0.000032
EC1       CARDPM12    0.000033
AVGGIFT   ETHC4       0.000036
RAMNTALL  POP901      0.000037
OCC12     CARDPM12    0.000041
TPE7      EIC6        0.000056
MARR4   

In [57]:
donors.loc[:,["HHD5","HHD12"]]

,HHD5,HHD12
0,84,3
1,95,2
2,86,3
3,84,4
4,92,1
...,...,...
95407,83,8
95408,96,1
95409,80,9
95410,84,6
